In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from tqdm.notebook import tqdm
import json
import glob
import os
import xarray as xr
from shapely.geometry import shape
import dask
import dask.dataframe as dd
import dask.array as da

In [2]:
client = dask.distributed.Client(n_workers=8)
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:59129,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:59853,Total threads: 1
Dashboard: http://127.0.0.1:59858/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:59141,


# Conversion to netcdf didn't work (below) - use dask DataFrame

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from tqdm.notebook import tqdm
import json
import glob
import os
import xarray as xr
from shapely.geometry import shape
import dask
import dask.dataframe as dd
import dask.array as da

In [15]:
def parse_item(item_fn):
    df = pd.read_csv(item_fn, low_memory=True)
    # coords = shape(json.loads(df['Coordinates'][0].replace("'", '"')))
    coords = df['Coordinates'][0].replace("'", '"') # replace ' with " to make it a valid wkt string
    product = df['Name'][4]
    location_name = df['Name'][0]

    if 'resultType' in df.columns:
        if df['resultType'].iloc[-1] == 'modelled':
            return None
    else:
        times = pd.to_datetime(df['dateTime'].iloc[9:].reindex(), utc=True)
        variable = df['result'].iloc[9:].reindex().apply(float)

        cleaned_dataset = pd.DataFrame({
            'time': pd.Series(times),
            f'values': pd.Series(variable, dtype=float),
            'location': location_name,
            'product': product,
            'geometry': coords
        }).reset_index().drop('index', axis='columns')#.set_index('time')
        
        return dd.from_pandas(cleaned_dataset, chunksize=5000)
        # return cleaned_dataset

In [16]:
r = parse_item('../data/rest/1/18.csv')
r

,time,values,location,product,geometry
npartitions=1,,,,,
0,"datetime64[ns, UTC]",float64,object,object,object
3731,...,...,...,...,...


In [17]:
r.head(10)

,time,values,location,product,geometry
0,1995-05-11 19:00:00+00:00,1362.11,Gray Reef Dam,Lake/Reservoir Storage,"{""type"": ""Point"", ""coordinates"": [-106.6989, 4..."
1,1995-05-12 19:00:00+00:00,1419.42,Gray Reef Dam,Lake/Reservoir Storage,"{""type"": ""Point"", ""coordinates"": [-106.6989, 4..."
2,1995-05-13 19:00:00+00:00,1171.44,Gray Reef Dam,Lake/Reservoir Storage,"{""type"": ""Point"", ""coordinates"": [-106.6989, 4..."
3,1995-05-14 19:00:00+00:00,1382.57,Gray Reef Dam,Lake/Reservoir Storage,"{""type"": ""Point"", ""coordinates"": [-106.6989, 4..."
4,1995-05-15 19:00:00+00:00,1650.08,Gray Reef Dam,Lake/Reservoir Storage,"{""type"": ""Point"", ""coordinates"": [-106.6989, 4..."
5,1995-05-16 19:00:00+00:00,1129.19,Gray Reef Dam,Lake/Reservoir Storage,"{""type"": ""Point"", ""coordinates"": [-106.6989, 4..."
6,1995-05-17 19:00:00+00:00,1096.38,Gray Reef Dam,Lake/Reservoir Storage,"{""type"": ""Point"", ""coordinates"": [-106.6989, 4..."
7,1995-05-18 19:00:00+00:00,1267.59,Gray Reef Dam,Lake/Reservoir Storage,"{""type"": ""Point"", ""coordinates"": [-106.6989, 4..."
8,1995-05-19 19:00:00+00:00,1488.19,Gray Reef Dam,Lake/Reservoir Storage,"{""type"": ""Point"", ""coordinates"": [-106.6989, 4..."
9,1995-05-20 19:00:00+00:00,1192.78,Gray Reef Dam,Lake/Reservoir Storage,"{""type"": ""Point"", ""coordinates"": [-106.6989, 4..."


In [18]:
cleaned_datasets = []
catalog_record_fns = glob.glob("../data/rest/*")
for catalog_record_fn in tqdm(catalog_record_fns):
    catalog_item_fns = glob.glob(os.path.join(catalog_record_fn, "*.csv"))
    for catalog_item_fn in catalog_item_fns:
        catalog_item = int(os.path.split(catalog_item_fn)[-1].split('.')[0])
        catalog_record = int(os.path.split(catalog_record_fn)[-1])

        cleaned_dataset = parse_item(catalog_item_fn)
        if cleaned_dataset is not None:
            cleaned_dataset['catalog_item'] = catalog_item
            cleaned_dataset['catalog_record'] = catalog_record

            cleaned_datasets.append(cleaned_dataset)

  0%|          | 0/166 [00:00<?, ?it/s]

/var/folders/2y/h464qnw52wq0lwvhs1s03dy40000gp/T/ipykernel_12863/3253778103.py:9: DtypeWarning: Columns (0,1,2,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  cleaned_dataset = parse_item(catalog_item_fn)
/var/folders/2y/h464qnw52wq0lwvhs1s03dy40000gp/T/ipykernel_12863/3253778103.py:9: DtypeWarning: Columns (0,1,2,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  cleaned_dataset = parse_item(catalog_item_fn)
/var/folders/2y/h464qnw52wq0lwvhs1s03dy40000gp/T/ipykernel_12863/3253778103.py:9: DtypeWarning: Columns (0,1,2,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  cleaned_dataset = parse_item(catalog_item_fn)
/var/folders/2y/h464qnw52wq0lwvhs1s03dy40000gp/T/ipykernel_12863/3253778103.py:9: DtypeWarning: Columns (0,1,2,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  cleaned_dataset = parse_item(catalog_item_fn)
/var/folders/2y/h464qnw52wq0lwvhs1s03dy40000gp/T/ipy

In [19]:
merged = dd.multi.concat(cleaned_datasets).reset_index().drop('index', axis='columns')
merged

,time,values,location,product,geometry,catalog_item,catalog_record
npartitions=1212,,,,,,,
,"datetime64[ns, UTC]",float64,object,object,object,int64,int64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [20]:
merged.to_csv('../data/US-RISE/*.csv', index=False)

['/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/../data/US-RISE/0000.csv',
 '/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/../data/US-RISE/0001.csv',
 '/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/../data/US-RISE/0002.csv',
 '/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/../data/US-RISE/0003.csv',
 '/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/../data/US-RISE/0004.csv',
 '/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/../data/US-RISE/0005.csv',
 '/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/../data/US-RISE/0006.csv',
 '/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/../data/US-RISE/0007.csv',
 '/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/../data/US-RISE/0008.csv',
 '/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/../data/US-RISE/0009.csv',
 '/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/../data/US-RISE/0010.csv',
 '/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/../data/US-RISE/0011.csv',
 '/Users/pdas47/phd/Ground-Truth/USA/RISE/notebooks/

# TODO: Make a netcdf file that has all the data

In [6]:
def parse_item(item_fn):
    df = pd.read_csv(item_fn)
    coords = shape(json.loads(df['Coordinates'][0].replace("'", '"')))
    product = df['Name'][4]
    location_name = df['Name'][0]

    if 'resultType' in df.columns:
        if df['resultType'].iloc[-1] == 'modelled':
            return None
        else:
            times = pd.to_datetime(df['dateTime'].iloc[9:].reindex(), utc=True)
            variable = df['result'].iloc[9:].reindex().apply(float)

            cleaned_dataset = pd.DataFrame({
                'time': times,
                f'values': variable,
                'location': location_name,
                'product': product,
                'geometry': coords
            })#.set_index('time')
            print(cleaned_dataset.info())
            # return dd.from_pandas(cleaned_dataset, chunksize=500)
            return cleaned_dataset

In [7]:
cleaned_datasets = []
catalog_record_fns = glob.glob("data/rest/*")
for catalog_record_fn in tqdm(catalog_record_fns):
    catalog_item_fns = glob.glob(os.path.join(catalog_record_fn, "*.csv"))
    for catalog_item_fn in catalog_item_fns:
        catalog_item = int(os.path.split(catalog_item_fn)[-1].split('.')[0])
        catalog_record = int(os.path.split(catalog_record_fn)[-1])

        cleaned_dataset = parse_item(catalog_item_fn)
        if cleaned_dataset is not None:
            cleaned_dataset['catalog_item'] = catalog_item
            cleaned_dataset['catalog_record'] = catalog_record

            cleaned_datasets.append(cleaned_dataset)

  0%|          | 0/166 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9118 entries, 9 to 9126
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   time      9118 non-null   datetime64[ns, UTC]
 1   values    8952 non-null   float64            
 2   location  9118 non-null   object             
 3   product   9118 non-null   object             
 4   geometry  9118 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(1), object(3)
memory usage: 356.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10084 entries, 9 to 10092
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   time      10084 non-null  datetime64[ns, UTC]
 1   values    9855 non-null   float64            
 2   location  10084 non-null  object             
 3   product   10084 non-null  object             
 4   geometry  10084 non-null  object   

/var/folders/2y/h464qnw52wq0lwvhs1s03dy40000gp/T/ipykernel_9979/2006743022.py:9: DtypeWarning: Columns (0,1,2,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  cleaned_dataset = parse_item(catalog_item_fn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10257 entries, 9 to 10265
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   time      10257 non-null  datetime64[ns, UTC]
 1   values    10025 non-null  float64            
 2   location  10257 non-null  object             
 3   product   10257 non-null  object             
 4   geometry  10257 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), object(3)
memory usage: 400.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4321 entries, 9 to 4329
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   time      4321 non-null   datetime64[ns, UTC]
 1   values    4316 non-null   float64            
 2   location  4321 non-null   object             
 3   product   4321 non-null   object             
 4   geometry  4321 non-null   object   

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [22]:
all_data = pd.concat(cleaned_datasets)

In [23]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7635387 entries, 9 to 17358
Data columns (total 7 columns):
 #   Column          Dtype              
---  ------          -----              
 0   time            datetime64[ns, UTC]
 1   values          float64            
 2   location        object             
 3   product         object             
 4   geometry        object             
 5   catalog_item    int64              
 6   catalog_record  int64              
dtypes: datetime64[ns, UTC](1), float64(1), int64(2), object(3)
memory usage: 466.0+ MB


In [39]:
# convert to netcdf:
#   variable: product
#   coords: record, item, time
#   attributes: location, geometry
staged = all_data.reset_index().drop('index', axis='columns').set_index(['catalog_record', 'catalog_item', 'time'])
print(staged)
staged_grouped = staged.groupby('product')

dataarrays = []

for key, group in tqdm(staged_grouped):
    location = group['location'].iloc[0]
    geometry = group['geometry'].iloc[0]
    # print(key, group.info(), group[group.index.duplicated()].info())
    data = group['values'].to_xarray().chunk({'time': 1000})
    data = data.rename(f'{key}')
    data.attrs['location'] = location
    data.attrs['geometry'] = geometry.wkt
    # dataarrays[key] = da
    dataarrays.append(data)


                                                          values  \
catalog_record catalog_item time                                   
4372           6136         1997-11-13 07:00:00+00:00  370377.00   
                            1997-11-14 07:00:00+00:00   17834.60   
                            1997-11-15 07:00:00+00:00   12374.70   
                            1997-11-16 07:00:00+00:00   11134.60   
                            1997-11-17 07:00:00+00:00   11943.70   
...                                                          ...   
2456           4483         2022-10-26 07:00:00+00:00     148.29   
                            2022-10-27 07:00:00+00:00    -889.55   
                            2022-10-28 07:00:00+00:00     296.47   
                            2022-10-29 07:00:00+00:00    -148.24   
                            2022-10-30 07:00:00+00:00    -148.23   

                                                                                         location  \
catalog_record

  0%|          | 0/21 [00:00<?, ?it/s]

In [40]:
dataarrays

[<xarray.DataArray 'Average San Juan Chama Inflow Volume' (catalog_record: 1, catalog_item: 1, time: 5051)>
 dask.array<xarray-<this-array>, shape=(1, 1, 5051), dtype=float64, chunksize=(1, 1, 1000), chunktype=numpy.ndarray>
 Coordinates:
   * catalog_record  (catalog_record) int64 2334
   * catalog_item    (catalog_item) int64 4503
   * time            (time) object 1230793200000000000 ... 1667113200000000000
 Attributes:
     location:  Heron Reservoir and Dam
     geometry:  POINT (-106.6992 36.6973),
 <xarray.DataArray 'Lake/Reservoir Area' (catalog_record: 46, catalog_item: 46, time: 19296)>
 dask.array<xarray-<this-array>, shape=(46, 46, 19296), dtype=float64, chunksize=(46, 46, 1000), chunktype=numpy.ndarray>
 Coordinates:
   * catalog_record  (catalog_record) int64 2422 2280 2289 ... 2291 2469 2456
   * catalog_item    (catalog_item) int64 4796 4810 4786 4807 ... 4817 4788 4811
   * time            (time) object 25200000000000 ... 1667113200000000000
 Attributes:
     location:

In [41]:
ds = xr.merge(dataarrays)

/Users/pdas47/opt/anaconda3/envs/sc/lib/python3.9/site-packages/xarray/core/indexing.py:1385: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
/Users/pdas47/opt/anaconda3/envs/sc/lib/python3.9/site-packages/xarray/core/indexing.py:1385: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexe

In [42]:
ds.chunk({'time': 1000})

<xarray.Dataset>
Dimensions:                                            (catalog_item: 557, catalog_record: 67, time: 19296)
Coordinates:
  * catalog_item                                       (catalog_item) int64 7...
  * catalog_record                                     (catalog_record) int64 ...
  * time                                               (time) object 25200000...
Data variables: (12/21)
    Average San Juan Chama Inflow Volume               (catalog_record, catalog_item, time) float64 dask.array<chunksize=(67, 557, 1000), meta=np.ndarray>
    Lake/Reservoir Area                                (catalog_record, catalog_item, time) float64 dask.array<chunksize=(67, 557, 1000), meta=np.ndarray>
    Lake/Reservoir Bank Storage                        (catalog_record, catalog_item, time) float64 dask.array<chunksize=(67, 557, 1000), meta=np.ndarray>
    Lake/Reservoir Change In Storage                   (catalog_record, catalog_item, time) float64 dask.array<chunksize=(67, 557, 1000), meta=np.ndarray>
    Lake/Reservoir Elevation                           (catalog_record, catalog_item, time) float64 dask.array<chunksize=(67, 557, 1000), meta=np.ndarray>
    Lake/Reservoir Evaporation                         (catalog_record, catalog_item, time) float64 dask.array<chunksize=(67, 557, 1000), meta=np.ndarray>
    ...                                                 ...
    Lake/Reservoir Side Inflow                         (catalog_record, catalog_item, time) float64 dask.array<chunksize=(67, 557, 1000), meta=np.ndarray>
    Lake/Reservoir Side Inflow Volume                  (catalog_record, catalog_item, time) float64 dask.array<chunksize=(67, 557, 1000), meta=np.ndarray>
    Lake/Reservoir Storage                             (catalog_record, catalog_item, time) float64 dask.array<chunksize=(67, 557, 1000), meta=np.ndarray>
    Stream Gage Height                                 (catalog_record, catalog_item, time) float64 dask.array<chunksize=(67, 557, 1000), meta=np.ndarray>
    Streamflow                                         (catalog_record, catalog_item, time) float64 dask.array<chunksize=(67, 557, 1000), meta=np.ndarray>
    Water Temperature                                  (catalog_record, catalog_item, time) float64 dask.array<chunksize=(67, 557, 1000), meta=np.ndarray>
Attributes:
    location:  Heron Reservoir and Dam
    geometry:  POINT (-106.6992 36.6973)

In [45]:
xr.save_mfdataset([ds], ['data/RISE-records.nc'])

KeyError: 'Lake/Reservoir Area'

In [ ]:
all_data.to_csv("data/US-RISE.csv")